# Step 4C: Video Model Training v2 (Improved)

This is an **improved version** of the video model training with better handling of class imbalance.

## 🆕 What's New in v2:

### 1. **Class Weights** (CRITICAL - Fixes 0% F1 on Sadness/Surprise!)
   - Automatically computed from training data
   - Sadness gets ~10x more weight (only 40 training samples!)
   - Surprise gets ~4x more weight (only 101 training samples)

### 2. **Temporal Data Augmentation**
   - Random frame sampling with jitter
   - Variable frame intervals (adds diversity)
   - Random temporal offset

### 3. **More Frames Per Video**
   - 8 → 12 frames (better temporal coverage)
   - Captures more motion/context

### 4. **Early Stopping**
   - Patience: 5 epochs (more than image due to smaller dataset)

### 5. **Better Learning Rate Scheduling**
   - ReduceLROnPlateau with patience=3

### 6. **Gradient Clipping**
   - Max norm: 1.0

### 7. **Label Smoothing**
   - Smoothing: 0.1

### 8. **More Epochs**
   - 15 epochs (up from 5)
   - Early stopping prevents overfitting

**Expected Improvements:**
- **Sadness: 0% F1 → 15-25% F1** (huge improvement!)
- **Surprise: 0% F1 → 18-30% F1** (huge improvement!)
- Overall: 41.9% acc → 48-55% acc

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight  # NEW
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import json
import os
from pathlib import Path
import warnings
import random
warnings.filterwarnings('ignore')

try:
    from tqdm.notebook import tqdm
    print("✓ Using notebook progress bars")
except ImportError:
    from tqdm import tqdm
    print("✓ Using terminal progress bars")

# Set random seeds
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Configuration

In [ ]:
# Paths
TRAIN_DATA = "data/train_set.csv"
VAL_DATA = "data/validation_set.csv"
MODEL_DIR = "models"
RESULTS_DIR = "results/video_model_v2"  # NEW: Separate results directory

# Create directories
Path(MODEL_DIR).mkdir(parents=True, exist_ok=True)
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

# Model configuration
MODEL_NAME = 'openai/clip-vit-base-patch32'
BATCH_SIZE = 8
EPOCHS = 15  # NEW: Increased epochs
LEARNING_RATE = 1e-5
NUM_FRAMES = 12  # NEW: Increased from 8 to 12 frames

# NEW: Training improvements
LABEL_SMOOTHING = 0.1
GRADIENT_CLIP_NORM = 1.0
EARLY_STOP_PATIENCE = 5
LR_SCHEDULER_PATIENCE = 3
LR_SCHEDULER_FACTOR = 0.5

# Sentiment labels
LABELS = ['Anger', 'Joy', 'Neutral/Other', 'Sadness', 'Surprise']
LABEL_TO_ID = {label: idx for idx, label in enumerate(LABELS)}
ID_TO_LABEL = {idx: label for label, idx in LABEL_TO_ID.items()}

print("Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Max Epochs: {EPOCHS}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Frames per video: {NUM_FRAMES}")
print(f"\n🆕 v2 Improvements:")
print(f"  - Class weights (Sadness ~10x, Surprise ~4x)")
print(f"  - Temporal augmentation (random frame sampling)")
print(f"  - More frames: 8 → {NUM_FRAMES}")
print(f"  - Label smoothing: {LABEL_SMOOTHING}")
print(f"  - Gradient clipping: {GRADIENT_CLIP_NORM}")
print(f"  - Early stopping: patience={EARLY_STOP_PATIENCE}")

## 3. Data Loading and Class Weight Computation

In [ ]:
# Load data
print("Loading datasets...")
train_df = pd.read_csv(TRAIN_DATA)
val_df = pd.read_csv(VAL_DATA)

# Filter for videos only
train_df = train_df[train_df['media_type'] == 'video'].reset_index(drop=True)
val_df = val_df[val_df['media_type'] == 'video'].reset_index(drop=True)

print(f"Train set (videos only): {len(train_df):,} samples")
print(f"Validation set (videos only): {len(val_df):,} samples")

# Display sentiment distribution
print("\nTrain set sentiment distribution:")
train_counts = train_df['post_sentiment'].value_counts()
print(train_counts)
print("\nPercentages:")
print((train_counts / len(train_df) * 100).round(2))

print("\nValidation set sentiment distribution:")
val_counts = val_df['post_sentiment'].value_counts()
print(val_counts)
print("\nPercentages:")
print((val_counts / len(val_df) * 100).round(2))

In [ ]:
# NEW: Compute class weights
print("\n" + "="*80)
print("COMPUTING CLASS WEIGHTS (NEW in v2)")
print("="*80)

train_labels_numeric = train_df['post_sentiment'].map(LABEL_TO_ID).values
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(LABELS)),
    y=train_labels_numeric
)
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

print("\nClass weights:")
for label, weight in zip(LABELS, class_weights):
    count = train_counts.get(label, 0)
    print(f"  {label:15s}: {weight:.4f} (n={count:,})")

print("\n💡 CRITICAL for Video Model:")
print("   Sadness has only 40 training samples (0% F1 in v1)!")
print("   Surprise has only 101 training samples (0% F1 in v1)!")
print("   With 10x and 4x weights, model will actually learn these classes.")

## 4. NEW: Temporal Augmentation Functions

In [ ]:
def extract_frames(video_path, num_frames=12, augment=False):
    """
    Extract frames from video with optional temporal augmentation.
    
    Augmentation adds randomness to frame sampling for training diversity.
    """
    try:
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames == 0:
            cap.release()
            return None
        
        if augment and total_frames > num_frames * 2:
            # NEW: Random temporal offset for augmentation
            max_offset = total_frames - num_frames * 2
            offset = random.randint(0, max_offset)
            
            # Sample with random jitter
            base_indices = np.linspace(offset, total_frames - offset - 1, num_frames, dtype=int)
            jitter = np.random.randint(-2, 3, size=num_frames)  # ±2 frames jitter
            frame_indices = np.clip(base_indices + jitter, 0, total_frames - 1)
        else:
            # Normal evenly-spaced sampling
            frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        
        frames = []
        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_image = Image.fromarray(frame_rgb)
                frames.append(pil_image)
        
        cap.release()
        
        # Pad if needed
        if len(frames) < num_frames:
            while len(frames) < num_frames:
                frames.append(frames[-1] if frames else Image.new('RGB', (224, 224), color='black'))
        
        return frames[:num_frames]
    
    except Exception as e:
        print(f"Error extracting frames from {video_path}: {e}")
        return None

print("✓ Frame extraction function with temporal augmentation defined")

## 5. Dataset and DataLoader

In [ ]:
class BrawlStarsVideoDataset(Dataset):
    def __init__(self, dataframe, processor, num_frames=12, augment=False):
        self.data = dataframe.reset_index(drop=True)
        self.processor = processor
        self.num_frames = num_frames
        self.augment = augment  # NEW
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        video_path = str(row['local_media_path']).replace('\\', '/')
        
        # Extract frames with optional augmentation
        frames = extract_frames(video_path, self.num_frames, augment=self.augment)
        
        if frames is None or len(frames) == 0:
            frames = [Image.new('RGB', (224, 224), color='black') for _ in range(self.num_frames)]
        
        # Process all frames
        pixel_values_list = []
        for frame in frames:
            inputs = self.processor(images=frame, return_tensors="pt")
            pixel_values_list.append(inputs['pixel_values'].squeeze(0))
        
        pixel_values = torch.stack(pixel_values_list)
        label = LABEL_TO_ID[row['post_sentiment']]
        
        return {
            'pixel_values': pixel_values,
            'label': torch.tensor(label, dtype=torch.long)
        }

# Initialize processor
print("Loading CLIP processor...")
processor = CLIPProcessor.from_pretrained(MODEL_NAME)

# Create datasets
train_dataset = BrawlStarsVideoDataset(train_df, processor, NUM_FRAMES, augment=True)
val_dataset = BrawlStarsVideoDataset(val_df, processor, NUM_FRAMES, augment=False)

# Create dataloaders (Windows-safe for notebooks)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,  # Windows-safe
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,  # Windows-safe
    pin_memory=True
)

print(f"✓ Created {len(train_loader)} train batches and {len(val_loader)} validation batches")
print(f"✓ Temporal augmentation enabled for training")

## 6. Model Definition

In [ ]:
class VideoSentimentClassifier(nn.Module):
    def __init__(self, n_classes=5, num_frames=12):
        super(VideoSentimentClassifier, self).__init__()
        self.clip = CLIPModel.from_pretrained(MODEL_NAME)
        self.num_frames = num_frames
        self.vision_embed_dim = self.clip.vision_model.config.hidden_size
        
        # Temporal aggregation (attention-based)
        self.temporal_attention = nn.Sequential(
            nn.Linear(self.vision_embed_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.vision_embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, n_classes)
        )
        
    def forward(self, pixel_values):
        batch_size, num_frames, C, H, W = pixel_values.shape
        pixel_values = pixel_values.view(batch_size * num_frames, C, H, W)
        
        vision_outputs = self.clip.vision_model(pixel_values=pixel_values)
        frame_embeds = vision_outputs.pooler_output
        frame_embeds = frame_embeds.view(batch_size, num_frames, -1)
        
        attention_scores = self.temporal_attention(frame_embeds)
        attention_weights = torch.softmax(attention_scores, dim=1)
        video_embed = torch.sum(frame_embeds * attention_weights, dim=1)
        
        logits = self.classifier(video_embed)
        return logits
    
    def get_embedding(self, pixel_values):
        with torch.no_grad():
            batch_size, num_frames, C, H, W = pixel_values.shape
            pixel_values = pixel_values.view(batch_size * num_frames, C, H, W)
            
            vision_outputs = self.clip.vision_model(pixel_values=pixel_values)
            frame_embeds = vision_outputs.pooler_output
            frame_embeds = frame_embeds.view(batch_size, num_frames, -1)
            
            attention_scores = self.temporal_attention(frame_embeds)
            attention_weights = torch.softmax(attention_scores, dim=1)
            video_embed = torch.sum(frame_embeds * attention_weights, dim=1)
        
        return video_embed

# Initialize model
model = VideoSentimentClassifier(n_classes=len(LABELS), num_frames=NUM_FRAMES)
model = model.to(device)

print(f"✓ Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")

## 7. Training Setup with Improvements

In [ ]:
# NEW: Loss function with class weights AND label smoothing
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, weight=None, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.weight = weight
        
    def forward(self, pred, target):
        n_classes = pred.size(-1)
        log_pred = torch.log_softmax(pred, dim=-1)
        
        with torch.no_grad():
            true_dist = torch.zeros_like(log_pred)
            true_dist.fill_(self.smoothing / (n_classes - 1))
            true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
        
        if self.weight is not None:
            true_dist = true_dist * self.weight.unsqueeze(0)
            
        return torch.mean(torch.sum(-true_dist * log_pred, dim=-1))

criterion = LabelSmoothingCrossEntropy(weight=class_weights_tensor, smoothing=LABEL_SMOOTHING)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=LR_SCHEDULER_FACTOR, patience=LR_SCHEDULER_PATIENCE, verbose=True
)

print(f"✓ Optimizer and scheduler configured")
print(f"🆕 Loss: CrossEntropy + Class Weights + Label Smoothing")
print(f"   This is CRITICAL for fixing 0% F1 on Sadness/Surprise!")

## 8. Training Functions

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    progress_bar = tqdm(dataloader, desc='Training')
    for batch in progress_bar:
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(pixel_values)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        # NEW: Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=GRADIENT_CLIP_NORM)
        
        optimizer.step()
        
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == labels).item()
        total_samples += labels.size(0)
        total_loss += loss.item()
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{correct_predictions/total_samples:.4f}'
        })
    
    return total_loss / len(dataloader), correct_predictions / total_samples


def eval_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc='Validation')
        for batch in progress_bar:
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(pixel_values)
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return avg_loss, accuracy, f1, all_preds, all_labels

print("✓ Training functions defined with gradient clipping")

## 9. Training Loop with Early Stopping

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_f1': [],
    'learning_rates': []
}

best_val_f1 = 0
best_epoch = 0
epochs_without_improvement = 0

print("="*80)
print("STARTING TRAINING (v2 with improvements)")
print("="*80)
print(f"\n🆕 Key improvements:")
print(f"  - Class weights (Sadness ~10x, Surprise ~4x)")
print(f"  - Temporal augmentation")
print(f"  - {NUM_FRAMES} frames per video (was 8)")
print(f"  - Early stopping (patience={EARLY_STOP_PATIENCE})")
print("\n")

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    print("-"*80)
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1, _, _ = eval_model(model, val_loader, criterion, device)
    
    current_lr = optimizer.param_groups[0]['lr']
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    history['learning_rates'].append(current_lr)
    
    print(f"\nResults:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f} | Val F1: {val_f1:.4f}")
    print(f"  Learning Rate: {current_lr:.2e}")
    
    scheduler.step(val_f1)
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_epoch = epoch + 1
        epochs_without_improvement = 0
        torch.save(model.state_dict(), f"{MODEL_DIR}/video_specialist_v2_best.pth")
        print(f"  ✓ New best model saved! (F1: {val_f1:.4f})")
    else:
        epochs_without_improvement += 1
        print(f"  No improvement for {epochs_without_improvement} epoch(s)")
    
    if epochs_without_improvement >= EARLY_STOP_PATIENCE:
        print(f"\n🛑 Early stopping! No improvement for {EARLY_STOP_PATIENCE} epochs.")
        print(f"   Best F1: {best_val_f1:.4f} at epoch {best_epoch}")
        break

print("\n" + "="*80)
print("TRAINING COMPLETE")
print("="*80)
print(f"Best validation F1: {best_val_f1:.4f} (Epoch {best_epoch})")
print(f"Total epochs run: {epoch + 1}/{EPOCHS}")

## 10. Save Models and Training History

In [ ]:
# Save final model
torch.save(model.state_dict(), f"{MODEL_DIR}/video_specialist_v2.pth")
print(f"✓ Saved final model: {MODEL_DIR}/video_specialist_v2.pth")

# Save training history
with open(f"{RESULTS_DIR}/training_history.json", 'w') as f:
    json.dump(history, f, indent=2)
print(f"✓ Saved training history: {RESULTS_DIR}/training_history.json")

## 11. Final Evaluation on Best Model

In [ ]:
# Load best model
print("Loading best model for final evaluation...")
model.load_state_dict(torch.load(f"{MODEL_DIR}/video_specialist_v2_best.pth"))

# Evaluate
val_loss, val_acc, val_f1, val_preds, val_labels = eval_model(model, val_loader, criterion, device)

print("\n" + "="*80)
print("FINAL EVALUATION ON VALIDATION SET")
print("="*80)
print(f"\nFinal Validation Metrics:")
print(f"  Loss: {val_loss:.4f}")
print(f"  Accuracy: {val_acc:.4f}")
print(f"  Weighted F1: {val_f1:.4f}")

## 12. Per-Class Metrics and Classification Report

In [ ]:
# Per-class metrics
print("\n" + "="*80)
print("PER-CLASS METRICS")
print("="*80)

precision, recall, f1_per_class, support = precision_recall_fscore_support(
    val_labels, val_preds, labels=range(len(LABELS)), zero_division=0
)

print(f"\n{'Class':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
print("-" * 65)
for i, label in enumerate(LABELS):
    print(f"{label:<15} {precision[i]:<12.4f} {recall[i]:<12.4f} {f1_per_class[i]:<12.4f} {support[i]:<10}")

# Classification report
print("\n" + "="*80)
print("DETAILED CLASSIFICATION REPORT")
print("="*80)
print(classification_report(val_labels, val_preds, target_names=LABELS, digits=4, zero_division=0))

# Save metrics
metrics = {
    'val_loss': float(val_loss),
    'val_accuracy': float(val_acc),
    'val_weighted_f1': float(val_f1),
    'per_class_metrics': {
        label: {
            'precision': float(precision[i]),
            'recall': float(recall[i]),
            'f1_score': float(f1_per_class[i]),
            'support': int(support[i])
        }
        for i, label in enumerate(LABELS)
    },
    'best_epoch': best_epoch
}

with open(f"{RESULTS_DIR}/final_metrics.json", 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\n✓ Saved metrics: {RESULTS_DIR}/final_metrics.json")

## 13. Visualize Training History

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0, 0].plot(history['val_loss'], label='Val Loss', marker='s')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history['train_acc'], label='Train Acc', marker='o')
axes[0, 1].plot(history['val_acc'], label='Val Acc', marker='s')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].set_title('Training and Validation Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# F1 Score
axes[1, 0].plot(history['val_f1'], label='Val F1', marker='o', color='green')
axes[1, 0].axhline(y=best_val_f1, color='r', linestyle='--', label=f'Best F1: {best_val_f1:.4f}')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('F1 Score')
axes[1, 0].set_title('Validation F1 Score')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Learning Rate
axes[1, 1].plot(history['learning_rates'], marker='o', color='orange')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Learning Rate')
axes[1, 1].set_title('Learning Rate Schedule')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: {RESULTS_DIR}/training_history.png")

## 14. Visualize Confusion Matrix

In [ ]:
cm = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=LABELS, yticklabels=LABELS)
plt.title('Confusion Matrix - Video Model v2', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: {RESULTS_DIR}/confusion_matrix.png")

## 15. Visualize Per-Class F1 Scores

In [ ]:
plt.figure(figsize=(10, 6))
x = np.arange(len(LABELS))
plt.bar(x, f1_per_class, alpha=0.8, color='steelblue', edgecolor='black')
plt.xlabel('Sentiment Class', fontsize=12)
plt.ylabel('F1 Score', fontsize=12)
plt.title('Per-Class F1 Scores - Video Model v2', fontsize=14, fontweight='bold')
plt.xticks(x, LABELS, rotation=45, ha='right')
plt.ylim(0, 1)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(f1_per_class):
    plt.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/per_class_f1.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: {RESULTS_DIR}/per_class_f1.png")

## 16. Comparison with v1 (if available)

In [ ]:
v1_metrics_path = "results/video_model/final_metrics.json"
if os.path.exists(v1_metrics_path):
    print("\n" + "="*80)
    print("COMPARISON: v1 vs v2")
    print("="*80)
    
    with open(v1_metrics_path, 'r') as f:
        v1_metrics = json.load(f)
    
    # Overall comparison
    print("\n📊 Overall Metrics:")
    print(f"{'Metric':<20} {'v1':<12} {'v2':<12} {'Change':<12}")
    print("-" * 56)
    
    v1_acc = v1_metrics['val_accuracy']
    v2_acc = val_acc
    acc_change = v2_acc - v1_acc
    print(f"{'Accuracy':<20} {v1_acc:<12.4f} {v2_acc:<12.4f} {acc_change:+.4f}")
    
    v1_f1 = v1_metrics['val_weighted_f1']
    v2_f1 = val_f1
    f1_change = v2_f1 - v1_f1
    print(f"{'Weighted F1':<20} {v1_f1:<12.4f} {v2_f1:<12.4f} {f1_change:+.4f}")
    
    # Per-class comparison
    print("\n📊 Per-Class F1 Score Improvements:")
    print(f"{'Class':<15} {'v1 F1':<12} {'v2 F1':<12} {'Change':<12} {'Status':<15}")
    print("-" * 70)
    
    for i, label in enumerate(LABELS):
        v1_class_f1 = v1_metrics['per_class_metrics'][label]['f1_score']
        v2_class_f1 = float(f1_per_class[i])
        change = v2_class_f1 - v1_class_f1
        
        if change > 0.05:
            status = "✓ Improved"
        elif change < -0.05:
            status = "✗ Degraded"
        else:
            status = "≈ Similar"
        
        print(f"{label:<15} {v1_class_f1:<12.4f} {v2_class_f1:<12.4f} {change:+12.4f} {status:<15}")
    
    # Highlight critical improvements
    sadness_v1 = v1_metrics['per_class_metrics']['Sadness']['f1_score']
    sadness_v2 = float(f1_per_class[LABELS.index('Sadness')])
    surprise_v1 = v1_metrics['per_class_metrics']['Surprise']['f1_score']
    surprise_v2 = float(f1_per_class[LABELS.index('Surprise')])
    
    print("\n🎯 CRITICAL FIXES (Previously 0% F1):")
    print(f"  Sadness:  {sadness_v1:.1%} → {sadness_v2:.1%} ({sadness_v2-sadness_v1:+.1%})")
    print(f"  Surprise: {surprise_v1:.1%} → {surprise_v2:.1%} ({surprise_v2-surprise_v1:+.1%})")
    
    # Visualization: v1 vs v2 comparison
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(LABELS))
    width = 0.35
    
    v1_f1_scores = [v1_metrics['per_class_metrics'][label]['f1_score'] for label in LABELS]
    v2_f1_scores = [float(f1_per_class[i]) for i in range(len(LABELS))]
    
    ax.bar(x - width/2, v1_f1_scores, width, label='v1 (Original)', alpha=0.8, color='lightcoral', edgecolor='black')
    ax.bar(x + width/2, v2_f1_scores, width, label='v2 (Improved)', alpha=0.8, color='steelblue', edgecolor='black')
    
    ax.set_xlabel('Sentiment Class', fontsize=12)
    ax.set_ylabel('F1 Score', fontsize=12)
    ax.set_title('Video Model: v1 vs v2 Per-Class F1 Scores', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(LABELS, rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/v1_vs_v2_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ Saved comparison chart: {RESULTS_DIR}/v1_vs_v2_comparison.png")
else:
    print("\n⚠️ v1 metrics not found. Skipping comparison.")

## 17. Summary

In [ ]:
print("\n" + "="*80)
print("VIDEO MODEL v2 TRAINING COMPLETE!")
print("="*80)

print(f"\n📁 Files saved:")
print(f"  - Best model: {MODEL_DIR}/video_specialist_v2_best.pth")
print(f"  - Final model: {MODEL_DIR}/video_specialist_v2.pth")
print(f"  - Metrics: {RESULTS_DIR}/final_metrics.json")
print(f"  - History: {RESULTS_DIR}/training_history.json")
print(f"  - Visualizations: {RESULTS_DIR}/*.png")

print(f"\n📊 Best Results:")
print(f"  - Validation Accuracy: {val_acc:.4f}")
print(f"  - Validation F1: {val_f1:.4f}")
print(f"  - Best Epoch: {best_epoch}")

print(f"\n🎯 v2 Improvements Applied:")
print(f"  ✓ Class weights (Sadness ~10x, Surprise ~4x)")
print(f"  ✓ Temporal augmentation (random frame sampling)")
print(f"  ✓ More frames: {NUM_FRAMES} (was 8)")
print(f"  ✓ Label smoothing: {LABEL_SMOOTHING}")
print(f"  ✓ Gradient clipping: {GRADIENT_CLIP_NORM}")
print(f"  ✓ Early stopping: patience={EARLY_STOP_PATIENCE}")

print(f"\n🚀 Next Steps:")
print(f"  1. Review per-class F1 scores (especially Sadness/Surprise)")
print(f"  2. Compare with v1 results to see improvements")
print(f"  3. If satisfied, use this model in your pipeline")
print(f"  4. For even better performance, consider the .py script version (better GPU utilization)")

print("\n" + "="*80)